### Описание:
Классифицировать игроков в каждом кадре по двум командам на основе цвета их формы.

### Данные:
Скачайте архив с данными [отсюда](https://disk.yandex.ru/d/NMlVHytktlzbEw) и распакуйте. Внутри архива лежит папка `frames` и файл `bboxes.json`. Файл `bboxes.json` содержит bounding boxes игроков двух команд, а в папке `frames` расположены соответствующие им изображения (кадры). Всего дано 100 кадров. Каждому кадру соответствует 10 bounding boxes игроков.

Структура файла `bboxes.json` следующая:
```
frame_n: {
     player_1: {
             'box': [x,y,w,h],
             'team': int
     }
     .....
     plyaer_10: { .... }
}
....
frame_n+k: { .... }
```
Здесь `frame_n` - номер кадра (ему соответствует файл с изображением в папке frames, всего 100),<br>
`player_1` - id игрока в текущем кадре,<br>
`'box': [x,y,w,h]` - bounding box соответствующего игрока (координаты нормализованы от 0 до 1, поэтому их нужно домножить на размеры изображения, чтобы перевести их в целые координаты фрейма),<br>
`'team': int` - id команды к которой принадлежит игрок  в данном кадре (может принимать значения 0 или 1).
Такм образом, id игроков и id команд имеют смысл только в пределах одного кадра и не связаны с другими кадрами. Т.е. в разных кадрах одному и тому же игроку / команде может соответствовать разные id.

### Задачи:
1) Классифицируйте игроков, используя в качестве вектора признаков игрока (features) средний цвет в пространстве RGB, который берется из области изображения, покрытой bounding box этого игрока (т.е. вектор признаков будет состоять из 3 элементов `[r,g,b]`.<br>
2) Используйте в качестве вектора признаков игроков их средний цвет в пространстве HSV (аналогично вектор признаков `[h,s,v]`.<br>
3) Используйте в качестве вектора признаков гистограмму в пространстве RGB, а не средний цвет. Число бинов определите самостоятельно.<br>
4) Используйте в качестве вектора признаков гистограмму в пространстве HSV.

### Как классифицировать:
В качестве классификатора можете использовать:
- Методы без обучения: k-means (на 2 кластера) или другие методы кластеризации.
- Методы с обучением: K-Nearest Neighbors, RandomForest, GaussianNB или др.

### Результаты:
1. Посчитайте точность классификации для четырех задач и сравните результаты. 
2. Задание будет принято, если итоговая точность будет выше чем 0.5 (рекорд на данный момент - 0.82)

In [1]:
import json
from os.path import join, abspath, basename
from glob import glob
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
plt.rcParams['figure.figsize'] = [15, 10]

сохраняем ббоксы и пути к фреймам

In [2]:
data_root = abspath('./team_classification_data/')
bboxes_path = join(data_root, 'bboxes.json')
frames_dir = join(data_root, 'frames')

with open(bboxes_path, 'r') as file:
    bboxes = json.load(file)
    
frames_paths = glob(join(frames_dir, '*.jpeg'))

In [3]:
# Проверка, что все кадры имеют одинаковый размер
shapes = set()
for p in frames_paths:
    img_bgr = cv.imread(p, cv.IMREAD_COLOR)
    shapes.update([img_bgr.shape])

assert len(shapes) == 1
print('img shape:', img_bgr.shape)
IMG_HEIGHT, IMG_WIDTH, _ = img_bgr.shape

img shape: (720, 1280, 3)


вспомогательные функции

In [4]:
def get_bbox(box):
    x,y,w,h = box
    return (int(x*IMG_WIDTH + 0.5), 
            int(y*IMG_HEIGHT + 0.5), 
            int(w*IMG_WIDTH + 0.5), 
            int(h*IMG_HEIGHT + 0.5))

def frame_id(path):
    return basename(path).rstrip('.jpeg')

def crop(img, xywh):
    x,y,w,h = xywh
    return img[y:y+h,x:x+w,:]

def plot(img_path):
    img_bgr = cv.imread(img_path, cv.IMREAD_COLOR)
    fig, ax = plt.subplots()
    ax.imshow(img_bgr[...,::-1])    
    for player in bboxes[frame_id(img_path)].values(): 
        if player['team'] == 0:
            x, y, w, h = bbox(player['box'])        
            rect = patches.Rectangle((x, y), w, h, edgecolor='r', facecolor='none')        
            ax.add_patch(rect)    
    plt.show()

def make_dataset(img_paths):    
    images = []
    ids = []
    for img_path in img_paths:
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        images.append(img)
        ids.append(frame_id(img_path))
        
    features, labels = [], []    
    for img, img_id in zip(images, ids):        
        for player in bboxes[img_id].values():
            player_img = crop(img, get_bbox(player['box']))
            features.append(player_img)
            labels.append(player['team'])
        
    return features, labels

In [ ]:
plot(frames_paths[0])

Специальный препроцессор, генерирующий из списка вырезанных по боксам картинок набор призников. Осуществляет препроцессинг: перевод в цветовое пространство, вычисление среднего по каналам или гистограмму, выравнивание гистограммы. Модель трансформера нужна для нормализации, чтоб только по обучающей выборке собираалсь статистика по среднему и ско, а применялась в трансформе. 

In [5]:
from sklearn.base import TransformerMixin, BaseEstimator

class FeatureMaker(TransformerMixin, BaseEstimator):
    
    def __init__(self, equalizeHist=False, n_bins=256, hsv=False, norm=False, feature_extractor='means'):        
        self.feature_extractor = feature_extractor            
        self.norm = norm
        self.hsv = hsv
        self.n_bins = n_bins
        self.equalizeHist = equalizeHist
    
    @staticmethod
    def equalize_hist(img):
        eq_img = [cv.equalizeHist(chanel) for chanel in np.rollaxis(img, 2)]
        return np.stack(eq_img, axis=2)
        
    @staticmethod
    def means(img):
        return img.mean(axis=(0,1))
    
    @staticmethod
    def hists(img, n_bins):
        hists = []
        for i in range(img.shape[2]):
            hist = cv.calcHist([img],[i],None,[n_bins],[0,256])
            hists.append(hist)
        return np.array(hists).flatten()

    def fit(self, X, y=None, **fit_params):
        
        new_X = []
        for x in X:
            if self.hsv:
                x = cv.cvtColor(x, cv.COLOR_BGR2HSV)
            
            if self.equalizeHist :
                x = self.equalize_hist(x)
            
            new_X.append(x)
            
        X = new_X
                
        if self.norm:
            _x = np.hstack([x.reshape(-1,3).T for x in X])
            self.mean = _x.mean(axis=1)
            self.std = _x.std(axis=1)
                              
        return self
                    
    def transform(self, X, y=None, **fit_params):
        features = []
        
        for x in X:
            
            if self.hsv:
                x = cv.cvtColor(x, cv.COLOR_BGR2HSV)
            
            if self.equalizeHist:
                x = self.equalize_hist(x)
                
            if self.norm:
                x = (x - self.mean) / self.std
                
            if self.feature_extractor == 'means':
                x = self.means(x)
            elif self.feature_extractor == 'hists':
                x = self.hists(x, self.n_bins)
            else:
                raise Exception('feature_extractor = means or hists')
                
            features.append(x)
        
        return features
    
        def fit_transform(self, X, y=None, **fit_params):
            return self.fit(X).transform(X)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.metrics import classification_report

разделение выборок на трейн и тест. Трейн будет применен в кросс-валидации

In [7]:
RANDOM_STATE = 1

X,y = make_dataset(frames_paths)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM_STATE, stratify=y)

Простой пайплайн: препроцесоср и градиентный бустинг. Рандомизированный поиск лучших моделей по сетке параметров с применением кросс-валидации.

In [8]:
preproc = FeatureMaker()
model = GradientBoostingClassifier(n_iter_no_change=10, random_state=RANDOM_STATE)
crossval = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=RANDOM_STATE)

pipe = Pipeline(steps=[("preproc", preproc), ("model", model)])

param_grid = {
    "preproc__norm": [True, False],
    "preproc__hsv": [True, False],
    "preproc__n_bins": (257 - np.logspace(3,0, 45)*0.255).astype(int),
    "preproc__equalizeHist": [True, False],   
    "preproc__feature_extractor": ['means', 'hists'],
    "model__loss": ['deviance', 'exponential'],
    "model__learning_rate": np.logspace(-1,-3, 15),
    "model__n_estimators": np.logspace(1,3, 15).astype(int)
}
search = RandomizedSearchCV(pipe, param_grid, n_iter=500, n_jobs=-1, scoring='accuracy', refit=True, cv=crossval)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.50777778 0.74888889 0.72333333 0.70444444        nan 0.78444444
 0.72666667 0.66              nan 0.66              nan 0.65
 0.69666667 0.70444444 0.69       0.68111111 0.69555556 0.79333333
        nan        nan 0.69              nan 0.65222222        nan
 0.84444444        nan        nan        nan        nan        nan
        nan 0.65444444 0.65888889 0.80888889 0.65777778 0.75555556
 0.7        0.70222222 0.60888889 0.70222222 0.65555556 0.77888889
 0.7        0.80888889 0.68       0.69666667 0.73555556 0.71888889
 0.68666667        nan 0.80333333        nan 0.69555556 0.70222222
 0.68666667 0.69666667 0.83              nan 0.65444444 0.71333333
        nan        nan 0.82222222 0.64444444 0.72333333 0.82777778
 0.68444444 0.79       0.70111111 0.80111111        nan 0.67777778
        nan 0.78              nan 0.78777778 0.69444444 0.81
 0

Best parameter (CV score=0.857):
{'preproc__norm': False, 'preproc__n_bins': 252, 'preproc__hsv': True, 'preproc__feature_extractor': 'hists', 'preproc__equalizeHist': False, 'model__n_estimators': 372, 'model__loss': 'deviance', 'model__learning_rate': 0.1}


In [11]:
import pandas as pd
from pprint import pprint

cv_results = pd.DataFrame(search.cv_results_).sort_values('mean_test_score', ascending=False)

In [12]:
means_res = cv_results.loc[:, "param_preproc__feature_extractor"] == 'means'
hist_res = cv_results.loc[:, "param_preproc__feature_extractor"] == 'hists'
bgr_res = cv_results.loc[:, "param_preproc__hsv"] == False
hsv_res = cv_results.loc[:, "param_preproc__hsv"] == True

Репорт на тесте

In [15]:
def report(conditions, params=None):
    mean_test_score = cv_results.loc[conditions]['mean_test_score'].iloc[0]
    if params is None:
        params = cv_results.loc[conditions]['params'].iloc[0]
    print('mean_test_score:', mean_test_score)
    print('\nbest params:')
    pprint(params)

    pipe.set_params(**params) 
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print('\nTEST Classification report:\n')
    print(classification_report(y_test, y_pred))

In [16]:
# 1) средний цвет в пространстве RGB
report(bgr_res & means_res)

mean_test_score: 0.7077777777777777

best params:
{'model__learning_rate': 0.0019306977288832496,
 'model__loss': 'deviance',
 'model__n_estimators': 51,
 'preproc__equalizeHist': False,
 'preproc__feature_extractor': 'means',
 'preproc__hsv': False,
 'preproc__n_bins': 256,
 'preproc__norm': True}

TEST Classification report:

              precision    recall  f1-score   support

           0       0.75      0.72      0.73        50
           1       0.73      0.76      0.75        50

    accuracy                           0.74       100
   macro avg       0.74      0.74      0.74       100
weighted avg       0.74      0.74      0.74       100



In [17]:
# 2) средний цвет в пространстве HSV (аналогично вектор признаков [h,s,v].
report(hsv_res & means_res)

mean_test_score: 0.7266666666666668

best params:
{'model__learning_rate': 0.019306977288832506,
 'model__loss': 'deviance',
 'model__n_estimators': 26,
 'preproc__equalizeHist': False,
 'preproc__feature_extractor': 'means',
 'preproc__hsv': True,
 'preproc__n_bins': 218,
 'preproc__norm': False}

TEST Classification report:

              precision    recall  f1-score   support

           0       0.78      0.62      0.69        50
           1       0.68      0.82      0.75        50

    accuracy                           0.72       100
   macro avg       0.73      0.72      0.72       100
weighted avg       0.73      0.72      0.72       100



In [18]:
# 3) Используйте в качестве вектора признаков гистограмму в пространстве RGB, 
report(bgr_res & hist_res)

mean_test_score: 0.8322222222222221

best params:
{'model__learning_rate': 0.1,
 'model__loss': 'exponential',
 'model__n_estimators': 26,
 'preproc__equalizeHist': False,
 'preproc__feature_extractor': 'hists',
 'preproc__hsv': False,
 'preproc__n_bins': 253,
 'preproc__norm': False}

TEST Classification report:

              precision    recall  f1-score   support

           0       0.88      0.74      0.80        50
           1       0.78      0.90      0.83        50

    accuracy                           0.82       100
   macro avg       0.83      0.82      0.82       100
weighted avg       0.83      0.82      0.82       100



In [19]:
# 4) Используйте в качестве вектора признаков гистограмму в пространстве HSV.
report(hsv_res & hist_res)

mean_test_score: 0.8566666666666667

best params:
{'model__learning_rate': 0.1,
 'model__loss': 'deviance',
 'model__n_estimators': 372,
 'preproc__equalizeHist': False,
 'preproc__feature_extractor': 'hists',
 'preproc__hsv': True,
 'preproc__n_bins': 252,
 'preproc__norm': False}

TEST Classification report:

              precision    recall  f1-score   support

           0       0.89      0.82      0.85        50
           1       0.83      0.90      0.87        50

    accuracy                           0.86       100
   macro avg       0.86      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100



# Результат

### Наилучшая точность достигнута при использовании в качестве вектора признаков гистограммы в пространстве HSV

# Accuracy 0.86

* На сбалансированной тестовой выборке из 100 сэмплов.